# Working with GPX-files in Python

By using numpy, pandas, SciKit-learn & gpxpy

a project by [Elian Van Cutsem](<http://www.elianvancutsem.com>)


## relative path to data-files

in the cell below you can define your own relative path to a folder with gpx files. This is the folder included in the repository.

In [ ]:
dataPath = './sampleData/'

### test data and representative parts

In the variables below you can choose either all the files in your gpx-file folder should be used or just a preview with a few files. The latter is way more performant but less precise. You can edit how many files should be used and choose to ignore the test data. These variables are used in further cells as well.

In [ ]:
import warnings

maxFilesToUse = 100
useTestData = True

# hide futureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### imports

before you run this cell, make sure you have installed the gpxpy library in anaconda by running `pip install gpxpy` in the anaconda prompt. The other libraries mentioned below are automatically installed with [anaconda](<https://www.anaconda.com/>)


In [ ]:
from collections import Counter
from datetime import timedelta
from matplotlib.pyplot import figure
from os import walk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from copy import copy

import os
import statistics
import gpxpy.gpx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn

# 1. Reading Data and visualising

## Reading the data

To edit and manipulate the given data, we first have to analyze how a gpx file is structured and can be used as a object in Python. In the first step below, we print out the files from the folder with the given gpx files. The code should be independent from any type of gpx file.

### Reading data from gpx files

The code below searches all files with a .gpx extension and puts them in a list. This list will later be used to effectively read data from the file with the name from the list. This step also makes use of the test data if you configured it. The test data variables will limit the number of files read by the code.

In [ ]:
gpxFileList = []
numberOfFiles = 1
numberOfFilesInDirectory = 0

for (path, names, filenames) in walk(dataPath):
    numberOfFilesInDirectory = len(filenames)
    
    for file in names:
        if file.endswith('.gpx'):
            if numberOfFiles >= maxFilesToUse and useTestData:
                break
            else:
                gpxFileList.append(file)
            numberOfFiles += 1
            
print(f"number of .gpx files: {len(gpxFileList)}/{numberOfFiles}")
print("total number of files in directory:", numberOfFilesInDirectory)
print("total number of files looked at:", numberOfFiles)

### Transforming read data to a dataframe

Now that we know how many files we can use and we have a variable list `gpxFileList` with all the filenames, every file in that list can be opened and transformed to a gpx usable object in Python. Now that we have to read and calculate the object. It's a perfect time to calculate some basic features to select usable and trustworthy files on. We will filter all unrepresentative data and files out so we only keep good files for further use. Since a gps is never 100% accurate, some files become completely unusable and are better ignored than used.
Since this step is quite intensive, it can take some time to calculate all the data. If it takes too long, you can limit the files in the variables initialized in the first few steps.

In [ ]:
def has_heartbeat(gpx):
    found_hr = 0
    for extensions in gpx.points[0].extensions:
        for extension in extensions:
            if extension.tag[-2:] == 'hr':
                found_hr = 1
                break
    return found_hr

In [ ]:
rawData = pd.DataFrame()

for file in gpxFileList:
    try:
        gpx_file = open(dataPath + file, 'r')
        gpx = gpxpy.parse(gpx_file)
        route = gpx.tracks[0]
        
        rawData = rawData.append({
            "filename": file,
            "file_id": file.split('_')[0],
            "creator": gpx.creator,
            "name": route.name,
            "date": route.get_time_bounds().start_time,
            "total distance": route.length_3d(),
            "total time": route.get_duration(),
            "nr of points": route.get_points_no(),
            "top speed": route.get_moving_data().max_speed,
            "has elevation": route.has_elevations(),
            "has heartbeat":  has_heartbeat(route.segments[0]),
            "type": file.split('_')[1][:-4],
        }, ignore_index=True)
    
    except:
        pass

rawData

## filtering Data

### Filtering of data

Here we're looping the results of the previous cell and we're going to check if they are usable or not. The variables that define if they're usable are shown in the cell below. All unusable files are removed from the dataframe.

In [ ]:
maxSpeed = 200
maxTime = timedelta(hours=24)
minTime = timedelta(minutes=5)
minNumberOfPoints = 8

In [ ]:
filteredData = pd.DataFrame()

for index, row in rawData.iterrows():
    error = 0
    try:
        if row['total time'] > maxTime.total_seconds():
            error = 1
        elif row['total time'] < minTime.total_seconds():
            error = 1
        elif row['number of points'] < minNumberOfPoints:
            error = 1
        elif row['top speed'] > maxSpeed:
            gpx_file = open(dataPath + row['filename'], 'r')
            gpx = gpxpy.parse(gpx_file)
            removedPoints = 0

            try:
                while gpx.get_moving_data().max_speed > maxSpeed:
                    points = gpx.tracks[0].segments[0].points
                    for i in range(0, len(points)-1):
                        punt = points[i-removedPoints]
                        nextPoint = points[i+1-removedPoints]
                        speed = punt.speed_between(nextPoint)
                        if speed > maxSpeed:
                            removedPoints += 1
                            del gpx.tracks[0].segments[0].points[i-removedPoints]
            except:
                error = 1
    except:
        error = 1
        
    if error == 0:
        filteredData = filteredData.append(row)

print('number of removed files:', str(len([name for name in os.listdir(dataPath)])-len(filteredData)))

filteredData = filteredData.reset_index()

filteredData

### Save data as a csv-file

Since it can take a lot of time to calculate all files, we make a csv file with all info we calculated so we can reuse all data without recalculating.

In [ ]:
filteredData.to_csv('filesTest.csv') if useTestData else filteredData.to_csv('files.csv')
print('Saved!')

## Looking at individual files

Now that only representative data is left, we can take a look at how a file is structured and what is stored in a GPX-file. We can translate all this info to charts.

### Reading the data from the previous made CSV-file

Here we read all data from the CSV-file that was previously created. This step can be skipped if all data was calculated on the same kernel. It never does harm to execute the cell below.

In [ ]:
filteredData = pd.read_csv('filesTest.csv') if useTestData else pd.read_csv('files.csv')
print('Read all data!')

### Interpretation of a random file

In the cell below we select a random file from the filtered data. Then we take a look at what info is stored the GPX object.

In [ ]:
rand = random.randrange(len(filteredData))
file = filteredData["filename"][rand]
gpx_file = open(dataPath + file, 'r')
gpx = gpxpy.parse(gpx_file)
print(gpx.to_xml())

In [ ]:
print("File: " +  filteredData["filename"][rand])
print("Creator of the track: " + gpx.creator)

for track in gpx.tracks:
    print("track name: " + track.name)
    print("number of points in the segment: ", gpx.get_points_no())
    print("length of the route " + str(round(gpx.length_2d() / 1000,2)) + 'km')
    print("first point in the segment: ", gpx.get_time_bounds().start_time)
    print("last point in the segment: ",  gpx.get_time_bounds().end_time)
    print("total time in seconds: ", track.get_duration())
        

### charts we can create by using a random file

If we have access to all points in a file, we can construct a chart which will match the route in the GPX object. Some files also have elevation data, speed data and heart rate data. If we have these, they also can be translated to a chart. (but not all files have these)

In [ ]:
lon = []
lat = []
height = []
speed = []
hr = []

print("File: " + file)
points = gpx.tracks[0].segments[0].points
for point in points:
    lon.append(float(point.longitude))
    lat.append(float(point.latitude))
    if point.elevation is not None:
        height.append(point.elevation)
    for extensions in point.extensions:
        for extension in extensions:
            if extension.tag[-5:] == 'speed':
                speed.append(float(extension.text))
            if extension.tag[-2:] == 'hr':
                            hr.append(float(extension.text))

fig, axs = plt.subplots(2,2, figsize=(15,15))

axs[0][0].semilogy(lon, lat)
axs[0][0].set_xlabel('longitude')
axs[0][0].set_ylabel('latitude')
axs[0][0].set_title('Route of the file {}'.format(file))
try:
    axs[0][1].semilogy(range(0, len(points)), height)
    axs[0][1].set_xlabel('number of points')
    axs[0][1].set_ylabel('height in m')
    axs[0][1].set_title('height-info of the file {}'.format(file))
except:
    print("this file doesn't contain any height data")
try:
    axs[1][0].semilogy(range(0, len(points)), speed)
    axs[1][0].set_xlabel('number of points')
    axs[1][0].set_ylabel('speed in km/u')
    axs[1][0].set_title('speed data of the file {}'.format(file))
except:
    print("this file doesn't contain any speed data")
try:
    axs[1][1].semilogy(range(0, len(points)), hr)
    axs[1][1].set_xlabel('number of points')
    axs[1][1].set_ylabel('Heart rate in bpm')
    axs[1][1].set_title('Heart rate data of the file {}'.format(file))
except:
    print("this file doesn't contain any heart rate data")

### simplifying the route

We can simplify the route using a point-reduction algorithm. This is the algorithm used:[Ramer-Douglas-Peucker algorithm](<http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm>)

In [ ]:
simpleGpx = copy(gpx)
simpleGpx.tracks[0].segments[0].simplify()
smoothLon = []
smoothLat = []

for punt in simpleGpx.tracks[0].segments[0].points:
    smoothLon.append(punt.longitude)
    smoothLat.append(punt.latitude)

fig, axs = plt.subplots(1,2, figsize=(16,8))
axs[0].semilogy(lon, lat)
axs[0].scatter(lon, lat, c='r')
axs[0].set_title('original points in route')
axs[1].semilogy(smoothLon, smoothLat)
axs[1].scatter(smoothLon, smoothLat, c='r')
axs[1].set_title('simplified points in route')
axs[0].set_xlabel('longitude')
axs[0].set_ylabel('latitude')
axs[1].set_xlabel('longitude')
axs[1].set_ylabel('latitude')

### comparing more random files

In the cell below we compare some random files. You can change the number of files used by changing the variable `numberOfFilesUsed`.

In [ ]:
numberOfFilesUsed = 8
numberOfPoints = 0
time = 0
heightInfo = 0
timeInfo = 0

for randomFile in range(numberOfFilesUsed):
    rand = random.randrange(len(filteredData)-1)
    file = filteredData["filename"][rand]
    gpx_file = open(dataPath + file, 'r')
    gpx = gpxpy.parse(gpx_file)
    time += gpx.get_duration()
    heightInfo += gpx.has_elevations()
    timeInfo += gpx.has_times()
    numberOfPoints += gpx.tracks[0].segments[0].get_points_no()
                
print("\nnumber of files analyzed: " + str(numberOfFilesUsed))
print("average time in a route: {} seconds".format(time/numberOfFilesUsed))
print("number of files with height info:", heightInfo)
print("number of files with time info: ", timeInfo)
print("average number of points in a file: " + str(numberOfPoints/numberOfFilesUsed) + "\n")

### info

below some charts about how the data is divided across the files.

#### representative data

In [ ]:
labels = ('representative data', 'not-representative data')
numbers = (len(filteredData), numberOfFiles-len(filteredData))

plt.bar(labels, numbers)
plt.ylabel('number of files')
plt.title('filtered data')

plt.show()

#### Height data

In [ ]:
numberOfFilesWithHeight = 0

for file in filteredData["has height"]:
    if file == 1:
        numberOfFilesWithHeight += 1
        
print('there are ' + str(numberOfFilesWithHeight) + ' files with height data out of {} files'.format(len(filteredData)))
print('that\'s ' + str(numberOfFilesWithHeight/len(filteredData)*100) + '%')

labels = ('files with height data', 'files without height data')
numbers = (numberOfFilesWithHeight, (len(filteredData)-numberOfFilesWithHeight))
y = labels

plt.bar(y, numbers)
plt.ylabel('number of files')
plt.title('number of files with height vs without height')
plt.show()

#### Heart rate data

In [ ]:
numberOfFilesWithHeartRate = 0

for file in filteredData["has heartbeat"]:
    if file > 0:
        numberOfFilesWithHeartRate += 1
        
print('there\'s {} files with height data on {} files'.format(numberOfFilesWithHeartRate,len(filteredData)))
print('that\'s {:.2f}%'.format(numberOfFilesWithHeartRate/len(filteredData)*100))

labels = ('files with heart rate', 'files without heart rate')
numbers = (numberOfFilesWithHeartRate, (len(filteredData)-numberOfFilesWithHeartRate))
y = labels

plt.bar(y, numbers)
plt.xticks(y, labels)
plt.ylabel('number of files')
plt.title('number of files with heart rate vs without heart rate')
plt.show()

## Distribution of the target

In this example we'll take the type of the route as target. This means that we'll make models to guess the type of the route by calculating other features. In the cells below is a study on available types.

### possible types

In the `routeTypes.txt` file in the example folder `datapath` we have a list of possible types and their meaning. Below is a rough print of the first 10 types in the file.

In [ ]:
types = pd.DataFrame(data=pd.read_csv(dataPath + "/routeTypes.txt", sep='\t', index_col=0))
types.head(10)

### number of files for each type

In [ ]:
counts = Counter(filteredData['type'])

labels = [types.type[int(label)] for label in counts]
figure(num=None, figsize=(6, 10), dpi=80, facecolor='w', edgecolor='k')
plt.pie([float(value) for value in counts.values()],
        labels=[str(value) + ' files' for value in counts.values()],
        autopct='%1.2f%%', colors='bgrcmy')
plt.title("types in original data")
plt.legend(labels, loc='upper center', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
plt.show()

print('here\'s another barchart:')
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
bars = plt.bar(labels, counts.values(), color='bgrcmy')
for bar in bars:
    yVal = bar.get_height()
    plt.text(bar.get_x() + 0.1, yVal, str(yVal) + ' files')
plt.title("number of files per type")
plt.xlabel("type")
plt.ylabel('number of files')


On this chart we can clearly see that not all types of the `routeTypes.txt` file are used. These types shouldn't be calculated and ignored, in the line of code below we filter out the types which aren't used and print only the used types.

In [ ]:
types['type'][[key for key in Counter(filteredData['type']).keys()]]

# 2. Deciding on features and calculating them

## Motivation on features

Here I've written some info on why features are (not) used

### used features:

 - **Total distance**: This is an easy to calculate feature. The gpxpy library does this for us. It's also a very important one since a type can be highly dependant on distance. For example; a pedestrian won't take a route as long as a car.
 - **Total time**: This feature is also calculated inside of the gpxpy feature. It's also a golden combo with the total distance.
 - **Top speed**: Top speed is also a function of the gpxpy library. It's a good feature since speed can give a lot of info about the vehicle used in the route.
 - **minimal speed**: The lowest speed possible is a usable feature, but can be 0 in a lot of cases. It's also highly correlating with other speed features. Nevertheless it can be important.
 - **minimal height**: This feature takes the lowest of all heights if there are any. Can be useful because speed and elevation are highly defining on a type.
 - **Total elevation**: Total elevation is also a function of the gpxpy library.
 - **average heart rate**: This function is only relevant in routes that contain this information. Probably is this a irrelevant function across all files because of the number of files that contain this data.
 - **total number of heartbeats**: This feature is highly correlating with other heart rate features, but still can be defining.
 - **average distance between 2 points**: average distance between two points in the route file. 
 - **Distance between the first and last point**: This is a feature I calculated myself. It can be useful to check if the route is circular or not.
 - **average number of points per kilometer**: speaks for itself.
 - **month of the track**: This feature I calculated myself. It can be very useful to define the type since a pedestrian or biker won't go out as often in winter as a driver.
 - **season**: This is a highly correlating feature with the month of a track, but this feature divides the year in 4 parts.
 - **starting-hour of the track**: This is a feature of the gpxpy library. It can be useful to check which types prefer morning or evening.
 - **Maximal heartbeat**: This feature checks for the highest heart rate in the file, but only 1/3rd of the files in the sample data contain heart rate data.
 - **Meridian of speed**: This feature checks all speeds of every point and calculates the meridian of all speeds.
 - **Standard deviation of speed**: This feature calculates the standard deviation of the speed, more info on how this works at [Standard deviation on wikipedia](<https://en.wikipedia.org/wiki/Standard_deviation>)
 - **Has cadence**: This extension can give the number of steps per minute. This can be a very good feature since it can give a very good clue on which type the route represents.


## calculating features

### code to calculate features (except for features provided by the gpxpy-library)

In [ ]:
def is_weekend(gpx):
    return 1 if gpx.points[0].time.weekday() >= 5 else 0

def get_low_speed(gpx):
    top_speed = gpx.get_moving_data().max_speed
    for i in range(len(gpx.points)-1):
        point = gpx.points[i]
        speed = point.speed_between(gpx.points[i + 1])
        if speed is None:
            return 0
        elif speed < top_speed:
            top_speed = speed
            
    return top_speed

def get_meridian_speed(gpx):
    speed = []
    for i in range(len(gpx.points)-1):
        point = gpx.points[i]
        speed.append(point.speed_between(gpx.points[i + 1]))
    try:
        return statistics.median(speed)
    except:
        return 0

def get_average_speed(gpx):
    n = len(gpx.points)
    speeds = []
    speed = 0
    
    for i in range(len(gpx.points)-1):
        point = gpx.points[i]
        temp_speed = point.speed_between(gpx.points[i + 1])
        if temp_speed is not None:
            speeds.append(point.speed_between(gpx.points[i + 1]))
            speed += point.speed_between(gpx.points[i + 1])
        else:
            n -= 1
    
    return speed / n
    
def get_deviation(gpx):
    n = len(gpx.points)
    speeds = []
    speed = 0
    variation = []
    variationNumber = 0
    
    for i in range(len(gpx.points)-1):
        point = gpx.points[i]
        temp_speed = point.speed_between(gpx.points[i + 1])
        if temp_speed is not None:
            speeds.append(point.speed_between(gpx.points[i + 1]))
            speed += point.speed_between(gpx.points[i + 1])
        else:
            n -= 1
    
    average = speed / n
    
    for speed in speeds:
        variation.append((speed - average)**2)
        
    for number in variation:
        variationNumber += number
        
    return variationNumber / (n-1)

def get_high_bpm(gpx):
    hr = 0
    for punt in gpx.points:
        for extensions in punt.extensions:
            for extension in extensions:
                if extension.tag[-2:] == 'hr':
                    if float(extension.text) > hr:
                        hr = float(extension.text)
    return hr

def get_low_bpm(gpx):
    hr = get_high_bpm(gpx)
    for punt in gpx.points:
        for extensions in punt.extensions:
            for extension in extensions:
                if extension.tag[-2:] == 'hr':
                    if float(extension.text) < hr:
                        hr = float(extension.text)
    return hr

def points_per_min(gpx):
    points = len(gpx.points)
    time = gpx.get_duration()
    if time is None or time == 0:
        return 0
    return points / time

def get_average_distance_between_points(gpx):
    distance = gpx.length_3d()
    return distance / gpx.get_points_no()

def get_start_to_end_distance(gpx):
    first_point = gpx.points[0]
    second_point = gpx.points[len(gpx.points)-1]
    distance = first_point.course_between(second_point)
    return distance

def has_cadence(gpx):
    found_cad = 0
    for extensions in gpx.points[0].extensions:
        for extension in extensions:
            if extension.tag[-3:] == 'cad':
                found_cad = 1
                break
    return found_cad

**Careful** These calculations can take some time to complete.

In [ ]:
features = pd.DataFrame()

for index, row in filteredData.iterrows():
    file = '{}_{}.gpx'.format(int(row['file_id']),int(row['type']))
    gpx_file = open(dataPath + file, 'r')
    gpx = gpxpy.parse(gpx_file)
    track = gpx.tracks[0].segments[0]

    features = features.append({
            "total distance": track.length_3d(),
            "total time": track.get_duration(),
            "type": row["type"],
            "top speed": track.get_moving_data().max_speed,
            "min speed": get_low_speed(track),
            "min elevation": track.get_elevation_extremes().minimum if track.get_elevation_extremes().minimum is not None else 0,
            "total elevation": track.get_uphill_downhill().uphill,
            "average distance between 2 points": get_average_speed(track),
            "distance between first and last point": get_start_to_end_distance(track),
            "average points per minute": points_per_min(track),
            "month": str(track.get_time_bounds().start_time.month),
            "season": str(int(track.get_time_bounds().start_time.month / 4)),
            "starting hour": str(track.get_time_bounds().start_time.hour),
            "isWeekend": is_weekend(track),
            "max heart rate": get_high_bpm(track),
            "meridian of speed": get_meridian_speed(track),
            "standard deviation in speed": get_deviation(track),
            "has cadence": has_cadence(track),
            "max elevation": track.get_elevation_extremes().maximum if track.get_elevation_extremes().maximum is not None else 0,
            "min heart rate": get_low_bpm(track),
            "average speed": get_average_speed(track),
        }, ignore_index=True)
    
features

## saving features as csv

In [ ]:
features.to_csv('featuresTest.csv') if useTestData else features.to_csv('features.csv')
print('saved!')

# 3. Feature evaluation and selection

## Reading features from csv-file

In [ ]:
features = pd.read_csv('featuresTest.csv') if useTestData else pd.read_csv('features.csv')
features = features.drop(['Unnamed: 0'], axis=1)
print('features read!')

## taking a look at the features

In [ ]:
features

## Correlation matrix

In [ ]:
correlationMatrix = features.drop(['type'], axis=1).corr()
plt.figure(figsize=(15,15))
plt.title('correlation matrix features')
seaborn.heatmap(correlationMatrix, annot=True, cmap="RdBu_r")
plt.show()

# 4. Constructing models

## setting up data/features and target

In [ ]:
y = features['type'].to_numpy().astype(int)
X = features.drop(['type'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=40, test_size=0.3)

In [ ]:
X

In [ ]:
print("Shape of data:", X.shape)
print("Shape of target:", y.shape)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("number of features:", X.shape[1])

## Selecting a model

Why to use a specific model

### KNN-model

KNN is the simplest model. It memorises the training-data and puts the to-predict data on the same chart. For each to predict target, it searches the nearest training point and sets the target value to the same as the neighbour. You can choose how many neighbours the model should include to give a value to the target.

### Decision tree model

A decision tree model can be very detailed, because of all the steps and decisions it takes, this can take more time than a KNN or Linear model

# 5. model evaluation and comparing

## Building models and taking a look

Below we start building models and take a quick look at them without parameter tuning.

### looking at the data

Taking a quick look at the data is very handy. We can see how each type is divided across the features, so we can select the best performing models to start parameter tuning.

In [ ]:
labels = [types.type[int(label)] for label in counts.keys()]
plot = pd.plotting.scatter_matrix(X, c=y, label=labels, figsize=(16, 10), marker='o', hist_kwds={'bins': 1})
plt.legend(labels, loc=1)
plt.show()

### using a KNN-model to define type

In [ ]:
neighbors = 15

In [ ]:
trainingScore = []
testScore = []

for n in range(1, neighbors+1):
    knn = KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)
    
    trainingScore.append(round(knn.score(X_train,y_train) * 100, 2))
    testScore.append(round(knn.score(X_test,y_test) * 100, 2))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
objects = range(1, neighbors+1)

ax1.bar(objects, trainingScore, align='center')
ax1.set_title('training score on ' + str(len(X)) + ' files')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('number of neighbors')

ax2.bar(objects, testScore, align='center')
ax2.set_ylim(0, 100)
ax2.set_title('test score on ' + str(len(X)) + ' files')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('number of neighbors')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
ax1.plot(objects, trainingScore)
ax1.set_title('enlarged scale impact per depth in training')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('number of neighbors')

ax2.plot(objects, testScore)
ax2.set_title('enlarged scale impact per depth in test')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('number of neighbors')
plt.show()

### using decision tree classifier to define type

In [ ]:
depth = 15

In [ ]:
trainingScore = []
testScore = []

for n in range(1, depth+1):
    treeModel = DecisionTreeClassifier(max_depth=n, random_state=42).fit(X_train, y_train)
    
    trainingScore.append(round(treeModel.score(X_train,y_train) * 100, 2))
    testScore.append(round(treeModel.score(X_test,y_test) * 100, 2))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
objects = range(1, depth+1)

ax1.bar(objects, trainingScore, align='center')
ax1.set_title('trainingScore on ' + str(len(X)) + ' files')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('depth')

ax2.bar(objects, testScore, align='center')
ax2.set_ylim(0, 100)
ax2.set_title('testScore on ' + str(len(X)) + ' files')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('depth')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
ax1.plot(objects, trainingScore)
ax1.set_title('enlarged scale impact per depth in in test')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('depth')

ax2.plot(objects, testScore)
ax2.set_title('enlarged scale impact per depth in test')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('depth')
plt.show()

### using random forest classifier to define type

In [ ]:
trainingScore = []
testScore = []

for n in range(1, depth+1):
    clf = RandomForestClassifier(n_estimators=n).fit(X_train,y_train)
    
    trainingScore.append(round(clf.score(X_train,y_train) * 100, 2))
    testScore.append(round(clf.score(X_test,y_test) * 100, 2))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
objects = range(1, depth+1)

ax1.bar(objects, trainingScore, align='center')
ax1.set_title('trainingScore on ' + str(len(X)) + ' files')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('number of estimators')

ax2.bar(objects, testScore, align='center')
ax2.set_ylim(0, 100)
ax2.set_title('testScore on ' + str(len(X)) + ' files')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('number of estimators')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
ax1.plot(objects, trainingScore)
ax1.set_title('enlarged scale per depth in training')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('number of estimators')

ax2.plot(objects, testScore)
ax2.set_title('enlarged scale per depth in training')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('number of estimators')
plt.show()

### Gradient Boosting classifier to define type

In [ ]:
trainingScore = []
testScore = []

depth = 10

for n in range(1, depth+1):
    gbrt = GradientBoostingClassifier(random_state=42, max_depth=n).fit(X_train, y_train)
    
    trainingScore.append(round(gbrt.score(X_train,y_train) * 100, 2))
    testScore.append(round(gbrt.score(X_test,y_test) * 100, 2))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
objects = range(1, depth+1)

ax1.bar(objects, trainingScore)
ax1.set_title('trainingScore on ' + str(len(X)) + ' files')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('depth ')

ax2.bar(objects, testScore, align='center')
ax2.set_ylim(0, 100)
ax2.set_title('testScore on ' + str(len(X)) + ' files')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('depth')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))
ax1.plot(objects, trainingScore)
ax1.set_title('enlarged scale per depth in training')
ax1.set_ylabel('percentage right (%)')
ax1.set_xlabel('depth')

ax2.plot(objects, testScore)
ax2.set_title('enlarged impact per depth in test')
ax2.set_ylabel('percentage right (%)')
ax2.set_xlabel('depth')
plt.show()

### logistic regression to define type

In [ ]:
logisticRegr = LogisticRegression(solver='liblinear', multi_class = 'auto')
logisticRegr.fit(X_train, y_train)

print('training score: ',round(logisticRegr.score(X_train,y_train) * 100, 2))
print('test score: ', round(logisticRegr.score(X_test,y_test) * 100, 2))

## tuning the models

### cross-validation

In [ ]:
kFold = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)

res_df = pd.DataFrame(cross_validate(knn, X, y, cv=kFold, return_train_score=True))
scores = cross_val_score(knn, X, y)

print("Average cross-validation score: {:.2f}".format(scores.mean()))
res_df

In [ ]:
lr = LinearRegression().fit(X_train, y_train)

res_df = pd.DataFrame(cross_validate(lr, X, y, cv=kFold, return_train_score=True))
scores = cross_val_score(lr, X, y)

print("Average cross-validation score: {:.2f}".format(scores.mean()))
res_df

In [ ]:
tree = DecisionTreeClassifier(max_depth=5, random_state=42).fit(X_train, y_train)

res_df = pd.DataFrame(cross_validate(tree, X, y, cv=kFold, return_train_score=True))
scores = cross_val_score(tree, X, y)

print("Average cross-validation score: {:.2f}".format(scores.mean()))
res_df

### Grid Search

Below we divide the data into a test-set, training-set and a validation set. These will be used to figure out which is the best model. After these steps, we use grid search to better the model.

In [ ]:
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_validation, y_train_validation, random_state=42)

print("Size of training set: {}   size of validation set: {}   size of test set:"
      " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

topScore = 0
bm = 0

#### GridSearch in KNN

In [ ]:
best_score = 0
best_parameters = 0

for n in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors = n).fit(X_train, y_train)
        score = knn.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_parameters = {'n_neighbors': n}
            if score > topScore:
                topScore = score
                bm = knn

print("Best parameters: ", best_parameters)

knn = KNeighborsClassifier(**best_parameters).fit(X_train_validation, y_train_validation)
training_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)

print("\nTraining set score with best parameters : {:.2f}".format(training_score))
print("Best score on validation set: {:.2f}".format(best_score))
print("Test set score with best parameters: {:.2f}".format(test_score))

#### GridSearch in Decision tree

In [ ]:
best_score = 0

for depth in range(1, 15):
    for rs in [0,5,10,20,42]:
        tree = DecisionTreeClassifier(max_depth=depth, random_state=rs)
        tree.fit(X_train, y_train)
        score = tree.score(X_valid, y_valid)
        
        if score > best_score:
            best_score = score
            best_parameters = {'max_depth': depth, 'random_state': rs}
            if score > topScore:
                topScore = score
                bm = tree

print("Best parameters: ", best_parameters)

tree = DecisionTreeClassifier(**best_parameters)
tree.fit(X_train_validation, y_train_validation)
training_score = tree.score(X_train, y_train)
test_score = tree.score(X_test, y_test)

print("\nTraining set score with best parameters : {:.2f}".format(training_score))
print("Best score on validation set: {:.2f}".format(best_score))
print("Test set score with best parameters: {:.2f}".format(test_score))

#### gridSearch in Random Forest

In [ ]:
best_score = 0

for n in range(1, 50):
    for rs in [0,5,10,20,42]:
        rfc = RandomForestClassifier(n_estimators=n, random_state=rs).fit(X_train, y_train)
        score = rfc.score(X_valid, y_valid)

        if score > best_score:
            best_score = score
            best_parameters = {'n_estimators': n, 'random_state':rs}
            if score > topScore:
                topScore = score
                bm = rfc

print("Best parameters: ", best_parameters)

rfc = RandomForestClassifier(**best_parameters).fit(X_train_validation, y_train_validation)
training_score = rfc.score(X_train, y_train)
test_score = rfc.score(X_test, y_test)

print("\nTraining set score with best parameters : {:.2f}".format(training_score))
print("Best score on validation set: {:.2f}".format(best_score))
print("Test set score with best parameters: {:.2f}".format(test_score))

#### gridSearch in Gradient Boosted Regression Trees

**careful** These calculations can take some time.

In [ ]:
best_score = 0

for n in range(1, 15):
        for rs in [0,5,10,20,42]:
            gbrt = GradientBoostingClassifier(random_state=rs, max_depth=n).fit(X_train, y_train)
            score = gbrt.score(X_valid, y_valid)

            if score > best_score:
                best_score = score
                best_parameters = {'max_depth': n, 'random_state': rs}
                if score > topScore:
                    topScore = score
                    bm = gbrt

print("Best parameters: ", best_parameters)

gbrt = RandomForestClassifier(**best_parameters).fit(X_train_validation, y_train_validation)
training_score = gbrt.score(X_train, y_train)
test_score = gbrt.score(X_test, y_test)

print("\nTraining set score with best parameters : {:.2f}".format(training_score))
print("Best score on validation set: {:.2f}".format(best_score))
print("Test set score with best parameters: {:.2f}".format(test_score))

# Best Model

Below is a printout of the best performing model, which is automatically selected based on the best test score. Further is also shown how the model relates into graphs.

In [ ]:
training_score = bm.score(X_train, y_train)
test_score = bm.score(X_test, y_test)
print("best model:", bm)
print("\nTraining set score with best parameters : {:.2f}".format(training_score))
print("Best score on validation set: {:.2f}".format(best_score))
print("Test set score with best parameters: {:.2f}".format(test_score))

Here's a graph on what features makes the biggest difference and are the most important to out model.

In [ ]:
def feature_importance(model):
    n_features = X.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), X)
    plt.title('Importance of features in model')
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

feature_importance(bm)

Below the predicted data and original data is put into a dataframe to later use in charts.

In [ ]:
results = bm.predict(X)
originalCount = Counter(y)
counts = Counter(results)
types = pd.DataFrame(data=pd.read_csv(dataPath + "/routeTypes.txt", sep='\t', index_col=0))

originalFrame = pd.DataFrame(dtype=int)

for result in originalCount:
    originalFrame = originalFrame.append({
       'type': result,
       'count': originalCount[result]
    }, ignore_index = True)

resultsFrame = pd.DataFrame(dtype=int)

for result in counts:
    resultsFrame = resultsFrame.append({
       'type': result,
       'count': counts[result]
    }, ignore_index = True)

originalFrame = originalFrame.sort_values('type')
resultsFrame = resultsFrame.sort_values('type')

## comparing original data vs predicted data

Below we display our final results from our best model. We can see how it performs.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,15))
labels = originalFrame['type']
ax1.pie(originalFrame['count'],
        labels=['{} files'.format(int(value)) for value in originalFrame['count']],
        autopct='%1.2f%%', colors='bgrcmy')
ax2.legend(labels, loc='upper center', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
ax1.set_title("types in original data")

labels = [types.type[int(label)] for label in resultsFrame['type']]
ax2.pie(resultsFrame['count'],
        labels=['{} files'.format(int(value)) for value in resultsFrame['count']],
        autopct='%1.2f%%', colors='bgrcmy')
ax2.legend(labels, loc='upper center', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
ax2.set_title("types in predicted data")

In [ ]:
print('here\'s another barchart comparing the original barchart')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))

labels = [types.type[int(label)] for label in originalFrame['type']]
bars = ax1.bar(labels, originalFrame['count'], color='bgrcmy')
for bar in bars:
    yVal = bar.get_height()
    ax1.text(bar.get_x() + 0.1, yVal, str(yVal) + ' files')
ax1.set_title("types in original data")
ax1.set_ylabel('number of files')
ax1.set_xlabel('types')

labels = [types.type[int(label)] for label in resultsFrame['type']]
bars = ax2.bar(labels, resultsFrame['count'], color='bgrcmy')
for bar in bars:
    yVal = bar.get_height()
    ax2.text(bar.get_x() + 0.1, yVal, str(yVal) + ' files')
ax2.set_title("types in predicted data")
ax2.set_ylabel('number of files')
ax2.set_xlabel('types')

In [ ]:
width =0.25

fig, ax = plt.subplots(figsize = (10,4))

ax.bar(np.arange(len(originalFrame['type'])), originalFrame['count'], width=width)
ax.bar(np.arange(len(resultsFrame['type']))+ width, resultsFrame['count'], width=width)
ax.set_xlabel('type')
ax.set_xticks(np.arange(len(originalFrame['type'])))
ax.set_xticklabels(labels)
ax.set_ylabel('number of files')
ax.legend(["original types", "predicted types"])
ax.set_title('original types vs predicted types')
plt.show()

# resolution

The predicted data clearly shows that the decision trees with tuned parameters ultimately score the best, especially the Random Forest and the Gradient boosted Regression score high. The data seems very well predicted. There is still some confusion between the "walk" type and the "recreational cycling" type. I eventually corrected this with the addition of the feature "hasCad", this feature consists of a binary variable that indicates whether the route contains a cadence. Cadence is expressed in steps per minute, so a cyclist can cycle as slowly as a walker, but will not have a cadence as high. All other features naturally also contribute to the decision tree.